# Imports

In [6]:
import numpy as np
import pandas as pd

# Dataset

In [94]:
small_labeled_data = pd.read_csv('resources/req_dataset.csv')
small_labeled_data = np.array(small_labeled_data)
print(small_labeled_data[0])

['stiff' 'https://api.github.com/users/stiff' 'https://github.com/stiff'
 'https://api.github.com/users/stiff/repos' 14 84 21 84
 "{'Python', 'CoffeeScript', 'Ruby', 'JavaScript'}" 21 0]


# Data Processing
- Get the relevant columns i.e., (4,6,7,9,11)

In [202]:
processed_labeled_data = np.zeros((3000, 5), dtype=int)
y = small_labeled_data[:, -1].astype(np.int64)
y.dtype

dtype('int64')

In [167]:

processed_labeled_data[:, :4] = small_labeled_data[:, (4,5,6,9)].astype(int)
processed_labeled_data[:][200]

array([172,  78,  19,  30,   0])

In [177]:
def language_count(garbage):
    top_tier_languages = {'Python','C', 'C++', 'JavaScript', 'TypeScript', 'Ruby', 'Rust', 'Lua', 'Assembly', 'Zig'}
    return len(set(garbage.replace(' ', '').replace('{', '').replace('}','').replace("\'", '').split(',')) & top_tier_languages)


In [194]:
processed_labeled_data[:, -1] = np.array([language_count(langs) for langs in small_labeled_data[:, 8]])
processed_labeled_data.shape
y.shape


(3000,)

In [205]:
y_label = np.zeros((len(y), len(np.unique(y))), dtype=np.int64)
y_label[np.arange(len(y)), y] = 1
y_label.shape


(3000, 2)

# Neural network


In [206]:
# @title Neural Network
class Layer_Dense:
  def __init__(self,n_inputs,n_neurons):
    self.weights = 0.01 * np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros((1,n_neurons))

  def forward(self,inputs):
    self.output = np.dot(inputs, self.weights)+ self.biases

  def parameters(self):
    return self.weights

class Activation_ReLU:
  def forward(self,inputs):
    self.output = np.maximum(0,inputs)

class Activation_Softmax:
  def forward(self,inputs):
    exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
    probabilities = exp_values/np.sum(exp_values,axis=1,keepdims=True)
    self.output = probabilities


class Loss_Categoricalcrossentropy():
  def forward(self, probs, y_true):
    samples = len(probs)
    probs_clipped = np.clip(probs, 1e-7, 1 - 1e-7)
    #negative_log_likelihoods = -np.sum(y_true*np.log(probs_clipped))
    #final_loss = np.sum(negative_log_likelihoods)/samples
    correct_confidences = np.sum(probs_clipped * y_true, axis=1)
    n_l_l = -np.log(correct_confidences)
    final_loss = np.mean(n_l_l)
    return final_loss


# Creating the layers and activations

In [210]:
layer1 = Layer_Dense(5,128)
layer2 = Layer_Dense(128,64)
layer3 = Layer_Dense(64,2)
activation1 = Activation_ReLU()
activation2 = Activation_ReLU()
activation3 = Activation_Softmax()

# Forward pass

In [253]:
layer1.forward(processed_labeled_data)
activation1.forward(layer1.output)
a1 = activation1.output
print(a1.shape)
layer2.forward(activation1.output)
activation2.forward(layer2.output)
a2 = activation2.output
print(a2.shape)
layer3.forward(activation2.output)
activation3.forward(layer3.output)
logits = activation3.output
print(logits.shape)

(3000, 128)
(3000, 64)
(3000, 2)


# Loss

In [254]:
loss_function = Loss_Categoricalcrossentropy()
loss = loss_function.forward(logits,y_label)
print(f"Loss : {loss}")

y_pred = np.argmax(activation3.output,axis=1)
accuracy = np.mean(y_pred == y)

print(f"Accuracy:{accuracy*100}")

Loss : 0.6777665437441145
Accuracy:59.599999999999994


# Backward pass

In [218]:
w1 = layer1.weights
w2 = layer2.weights
w3 = layer3.weights
dw3 = a2.T @ (logits - y_label)
db3 = np.sum((logits - y_label),axis=0)
da2 = (logits - y_label) @ w3.T
dz2 = da2 * (a2 > 0)
dw2 = a1.T @ dz2
db2 = np.sum(dz2,axis=0)
da1 = dz2 @ w2.T
dz1 = da1 * (a1 > 0)
dw1 = processed_labeled_data.T @ dz1
db1 = np.sum(dz1,axis=0)

# Gradient descent

In [252]:

learning_rate = 0.001
layer1.weights -= learning_rate * dw1
layer2.weights -= learning_rate * dw2
layer3.weights -= learning_rate * dw3
layer1.biases -= learning_rate * db1
layer2.biases -= learning_rate * db2
layer3.biases -= learning_rate * db3

# Training loop